In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install -U sentence-transformers
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from sentence_transformers import SentenceTransformer, util
import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 932.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=d4d75bed578f7af8c26eb9aa764c68c1fa3e52d4abafe5410586624da8646a3f
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [2]:
en_quran = pd.read_csv('../input/quran-english/Quran_English.csv')
en_torah = pd.read_csv('../input/the-old-testament-of-the-king-james-version/EN_old_testament_of_kjv.csv')
en_quran.head(1)

,Name,Surah,Ayat,Verse
0,The Opening,1,1,"In the name of Allah, the Beneficent, the Merc..."


In [3]:
en_torah.head(1)

,citation,book,chapter,verse,text
0,Genesis 1:1,Genesis,1,1,In the beginning God created the heaven and th...


In [4]:
max_len = en_quran.Verse.str.split().map(lambda x: len(x))
max_len = max_len.tolist()
max_val =  max(max_len)
print(max_val)
max_idx = max_len.index(max_val)
max_idx

273


288

In [5]:
en_torah.text[10]

'And God said, Let the earth bring forth grass, the herb yielding seed, and the fruit tree yielding fruit after his kind, whose seed is in itself, upon the earth: and it was so. \n'

In [6]:
max_len = en_torah.text.str.split().map(lambda x: len(x))
max_len = max_len.tolist()
max_val =  max(max_len)
print(max_val)
max_idx = max_len.index(max_val)
max_idx

86


12826

In [7]:
torah_list = en_torah.text.tolist()
quran_list = en_quran.Verse.tolist()
len(torah_list),len(quran_list)

(23145, 6236)

In [8]:
#asymmetric semantic search
model = SentenceTransformer('msmarco-distilbert-base-tas-b', device="cuda")


#Compute embedding for both lists (quran and torah)
torah_embeddings = model.encode(torah_list, convert_to_tensor=True)
quran_embeddings = model.encode(quran_list, convert_to_tensor=True)

torah_embeddings = torah_embeddings.to('cuda')
torah_embeddings = util.normalize_embeddings(torah_embeddings)

quran_embeddings = quran_embeddings.to('cuda')
quran_embeddings = util.normalize_embeddings(quran_embeddings)


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/547 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Batches:   0%|          | 0/724 [00:00<?, ?it/s]

Batches:   0%|          | 0/195 [00:00<?, ?it/s]

In [9]:
len(torah_embeddings),len(quran_embeddings)

(23145, 6236)

In [10]:
%%time

hits = util.semantic_search(torah_embeddings, quran_embeddings, score_function=util.dot_score)

dot_score = util.dot_score(torah_embeddings[0], quran_embeddings[0]).item()

dot_score


CPU times: user 258 ms, sys: 29.1 ms, total: 287 ms
Wall time: 292 ms


0.7763976454734802

In [11]:
sim_quran_torah = en_torah.copy()
sim_quran_torah = sim_quran_torah.rename(columns={"citation": "t_citation", "book": "t_book", "chapter": "t_chapter", "verse": "t_verse", "text": "t_text"})


sim_quran_torah['q_Name'] = "Quran Surah Name"
sim_quran_torah['q_Surah'] = "Quran Surah Number"
sim_quran_torah['q_Ayat'] = "Quran Ayat Number of corresponding Surah"
sim_quran_torah['q_Verse'] = "Quranic Verse of Corresponding surah and ayat"

sim_quran_torah['similarity_score'] = 0.0

sim_quran_torah.head(1)

,t_citation,t_book,t_chapter,t_verse,t_text,q_Name,q_Surah,q_Ayat,q_Verse,similarity_score
0,Genesis 1:1,Genesis,1,1,In the beginning God created the heaven and th...,Quran Surah Name,Quran Surah Number,Quran Ayat Number of corresponding Surah,Quranic Verse of Corresponding surah and ayat,0.0


In [12]:
%%time

for i in range(len(torah_embeddings)):
    best_score = 0.0
    idx = 0
    for j in range(len(quran_embeddings)):
        dot_score = util.dot_score(torah_embeddings[i], quran_embeddings[j]).item()
        if(dot_score>best_score):
            best_score = dot_score
            idx = j
            
 
    sim_quran_torah.loc[i, 'similarity_score'] = best_score
    sim_quran_torah.loc[i, 'q_Name'] = en_quran.loc[idx, 'Name']
    sim_quran_torah.loc[i, 'q_Surah'] = en_quran.loc[idx, 'Surah']
    sim_quran_torah.loc[i, 'q_Ayat'] = en_quran.loc[idx, 'Ayat']
    sim_quran_torah.loc[i, 'q_Verse'] = en_quran.loc[idx, 'Verse']
    
sim_quran_torah.head()

CPU times: user 1h 29min 23s, sys: 4.66 s, total: 1h 29min 28s
Wall time: 1h 29min 47s


,t_citation,t_book,t_chapter,t_verse,t_text,q_Name,q_Surah,q_Ayat,q_Verse,similarity_score
0,Genesis 1:1,Genesis,1,1,In the beginning God created the heaven and th...,Tâ Hâ,20,4,A revelation from Him Who created the earth an...,0.905833
1,Genesis 1:2,Genesis,1,2,"And the earth was without form, and void; and ...",The Story,28,40,"Therefor We seized him and his hosts, and aban...",0.852455
2,Genesis 1:3,Genesis,1,3,"And God said, Let there be light: and there wa...",Women,4,174,O mankind! Now hath a proof from your Lord com...,0.859681
3,Genesis 1:4,Genesis,1,4,"And God saw the light, that it was good: and G...",The Poets,26,28,He said: Lord of the East and the West and all...,0.857810
4,Genesis 1:5,Genesis,1,5,"And God called the light Day, and the darkness...",Those Who Drag Forth,79,29,"And He made dark the night thereof, and He bro...",0.865474


In [13]:

sim_quran_torah.sort_values(by=['similarity_score'], ascending=False)

,t_citation,t_book,t_chapter,t_verse,t_text,q_Name,q_Surah,q_Ayat,q_Verse,similarity_score
1260,Genesis 42:8,Genesis,42,8,"And Joseph knew his brethren, but they knew no...",Joseph,12,58,And Joseph’s brethren came and presented thems...,0.966047
5099,Deuteronomy 6:13,Deuteronomy,6,13,"Thou shalt fear the LORD thy God, and serve hi...",Those Who Drag Forth,79,19,Then I will guide thee to thy Lord and thou sh...,0.946708
1772,Exodus 9:30,Exodus,9,30,"But as for thee and thy servants, I know that ...",Those Who Drag Forth,79,19,Then I will guide thee to thy Lord and thou sh...,0.941159
14329,Psalms 30:10,Psalms,30,10,"Hear, O LORD, and have mercy upon me: LORD, be...",Smoke,44,6,"A mercy from thy Lord. Lo! He, even He is the ...",0.938172
20227,Jeremiah 51:15,Jeremiah,51,15,"He hath made the earth by his power, he hath e...",The Bee,16,3,He hath created the heavens and the earth with...,0.936844
...,...,...,...,...,...,...,...,...,...,...
12795,Esther 6:2,Esther,6,2,"And it was found written, that Mordecai had to...",Those Who Drag Forth,79,29,"And He made dark the night thereof, and He bro...",0.760951
17596,Song of Solomon 4:14,Song of Solomon,4,14,"Spikenard and saffron; calamus and cinnamon, w...",The Beneficent,55,58,(In beauty) like the jacynth and the coral-stone.,0.760915
21140,Ezekiel 27:19,Ezekiel,27,19,Dan also and Javan going to and fro occupied i...,The Cave,18,70,"He said: Well, if thou go with me, ask me not ...",0.760534
6181,Joshua 13:27,Joshua,13,27,"And in the valley, Betharam, and Bethnimrah, a...",The Story,28,30,"And when he reached it, he was called from the...",0.757143


In [14]:
sim_quran_torah.to_csv("similarities_between_quran_and_torah.csv",index = False)